# Well Test Analysis and Productivity Index Optimization

This notebook teaches you how to analyze well test data and optimize productivity index (PI) and reservoir pressure parameters using engineering methods. You'll learn to evaluate different mathematical models and understand which ones provide reliable results for field applications.

**What You'll Learn:**
- How to analyze well test data systematically
- Methods for calculating productivity index and reservoir pressure
- How to evaluate model accuracy using engineering criteria
- When to trust mathematical models for field decisions

## Step 1: Setup Required Tools

Run the cell below to install the mathematical and plotting tools we need. Google Colab usually has these already installed, but this ensures everything is ready.

In [ ]:
# Install runtime dependencies (Colab) - uncomment if needed
# !pip install pandas numpy matplotlib scipy openpyxl

## Step 2: Load Mathematical Tools

Import the tools we'll use for calculations, data analysis, and creating plots. Think of this as getting your calculator, spreadsheet, and graphing tools ready.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime
import io, os

## Step 3: Well Test Analysis Engine

This section contains the mathematical engine that analyzes well test data. The engine can:

- Calculate flow rates using different reservoir models (Linear PI, Vogel's equations)
- Automatically choose the right model based on pressure conditions
- Find the best-fit parameters for your well data
- Evaluate how reliable the results are using multiple accuracy measures

**Key Concept**: We use multiple models because wells behave differently depending on whether they produce single-phase or two-phase flow, and whether the reservoir pressure is above or below the bubble point.

In [ ]:
class WellTestOptimizer:
    """
    Well Test Data Optimizer for PI and SBHP (Notebook-friendly)
    
    Enhanced with multi-criteria fit quality assessment (v1.2):
    - RMSE and relative error % for practical engineering decisions
    - R² as supplementary metric (can mislead with low-variance data)
    - Improved correlation checks for small groups
    - Robust optimization with multiple methods
    """
    def __init__(self):
        self.data = None
        self.results = []

    def calculate_ipr(self, fbhp, sbhp, pbub, pi):
        # enforce non-negative inputs
        fbhp = max(0.0, float(fbhp))
        sbhp = max(0.0, float(sbhp))
        pbub = max(0.0, float(pbub))
        pi = max(0.0, float(pi))

        # Physical: if FBHP > SBHP, no production
        if fbhp > sbhp:
            return 0.0

        # Case 1: Linear PI (single-phase) - FBHP >= Pbub and SBHP > Pbub
        if (fbhp >= pbub) and (sbhp > pbub):
            ql = pi * (sbhp - fbhp)
        # Case 2: Modified Vogel (unsaturated reservoir) - FBHP < Pbub and SBHP > Pbub
        elif (fbhp < pbub) and (sbhp > pbub):
            ql_at_pbub = pi * (sbhp - pbub)
            if pbub > 0.0:
                fbhp_pbub_ratio = fbhp / pbub
                vogel_component = (pi * pbub / 1.8) * (1 - 0.2 * fbhp_pbub_ratio - 0.8 * fbhp_pbub_ratio**2)
                ql = ql_at_pbub + vogel_component
            else:
                ql = ql_at_pbub
        # Case 3: Standard Vogel (saturated reservoir) - SBHP <= Pbub
        else:
            if sbhp > 0.0:
                fbhp_sbhp_ratio = fbhp / sbhp
                ql = (pi * sbhp / 1.8) * (1 - 0.2 * fbhp_sbhp_ratio - 0.8 * fbhp_sbhp_ratio**2)
            else:
                ql = 0.0

        return max(0.0, ql)

    def calculate_ql_max(self, sbhp, pbub, pi):
        return self.calculate_ipr(0.0, sbhp, pbub, pi)

    def objective_function(self, params, fbhp_data, q_actual, pbub):
        sbhp, pi = params
        # penalty for non-physical parameters
        if sbhp <= 0 or pi <= 0:
            return 1e12
        max_fbhp = np.max(fbhp_data)
        if sbhp <= max_fbhp:
            return 1e12

        q_calculated = np.array([self.calculate_ipr(f, sbhp, pbub, pi) for f in fbhp_data])
        if np.any(q_calculated < 0):
            return 1e12

        sse = np.sum((q_actual - q_calculated)**2)
        # soft penalty for extreme values
        if sbhp > 15000 or pi > 1000:
            sse += 1e8
        return sse

    def optimize_group(self, group_data, pbub, sbhp_guess, pi_guess):
        fbhp_data = group_data['Pwf'].values
        q_actual = group_data['Total Rate'].values

        # data checks
        if np.any(fbhp_data <= 0) or np.any(q_actual <= 0):
            return {'success': False, 'message': 'Non-positive data values detected'}

        # Enhanced correlation check - skip for small groups (≤3 points)
        try:
            fbhp_variance = np.var(fbhp_data)
            rate_variance = np.var(q_actual)
            
            if fbhp_variance < 1e-10 or rate_variance < 1e-10:
                fbhp_rate_correlation = 0.0
            else:
                correlation_matrix = np.corrcoef(fbhp_data, q_actual)
                fbhp_rate_correlation = correlation_matrix[0, 1]
                if np.isnan(fbhp_rate_correlation):
                    fbhp_rate_correlation = 0.0
            
            # For small groups (≤3 points), skip correlation check as it's unreliable
            if len(group_data) > 3 and fbhp_rate_correlation > 0.9:
                return {'success': False, 'message': f'Data quality issue: very strong positive FBHP-rate correlation ({fbhp_rate_correlation:.3f}).'}
        except Exception:
            fbhp_rate_correlation = 0.0

        # More flexible bounds for better convergence
        max_fbhp = np.max(fbhp_data)
        sbhp_lower = max_fbhp + 50    # Reduced minimum drawdown
        sbhp_upper = min(10000.0, max_fbhp + 5000.0)  # Increased upper limit

        rate_range = np.max(q_actual) - np.min(q_actual)
        fbhp_range = np.max(fbhp_data) - np.min(fbhp_data)
        estimated_pi = (rate_range / fbhp_range) if fbhp_range > 1 else 1.0  # Relaxed threshold
        pi_lower = max(0.001, estimated_pi * 0.01)   # Much wider range
        pi_upper = min(100.0, estimated_pi * 100)    # Much wider range

        # Adjust initial guesses to bounds
        sbhp_guess = float(np.clip(sbhp_guess, sbhp_lower + 100, sbhp_upper - 100)) if sbhp_upper - sbhp_lower > 200 else (sbhp_lower + 200.0)
        pi_guess = float(np.clip(pi_guess, pi_lower, pi_upper))

        initial_guess = [sbhp_guess, pi_guess]
        bounds = [(sbhp_lower, sbhp_upper), (pi_lower, pi_upper)]

        # Try multiple optimization methods for robustness
        result = None
        methods = ['L-BFGS-B', 'TNC', 'SLSQP']
        
        for method in methods:
            try:
                result = minimize(self.objective_function, initial_guess, 
                                args=(fbhp_data, q_actual, pbub), method=method, bounds=bounds)
                if result.success:
                    break
            except Exception:
                continue
        
        # If all methods failed, try with relaxed bounds
        if result is None or not result.success:
            try:
                relaxed_bounds = [(sbhp_lower - 50, sbhp_upper + 1000), 
                                (pi_lower * 0.1, pi_upper * 2)]
                result = minimize(self.objective_function, initial_guess,
                                args=(fbhp_data, q_actual, pbub), method='L-BFGS-B', bounds=relaxed_bounds)
            except Exception:
                return {'success': False, 'message': 'All optimization methods failed'}

        if result and result.success:
            sbhp_opt, pi_opt = result.x
            q_calculated = np.array([self.calculate_ipr(f, sbhp_opt, pbub, pi_opt) for f in fbhp_data])
            
            # Enhanced metrics calculation
            ss_res = np.sum((q_actual - q_calculated)**2)
            ss_tot = np.sum((q_actual - np.mean(q_actual))**2)
            
            # Calculate comprehensive metrics
            if ss_tot == 0:
                r_squared = 0.0
                rmse = np.sqrt(np.mean((q_actual - q_calculated)**2))
                fit_quality = "identical_data"
            else:
                r_squared = 1 - (ss_res / ss_tot)
                rmse = np.sqrt(ss_res / len(q_actual))
            
            # Additional metrics
            mean_abs_error = np.mean(np.abs(q_actual - q_calculated))
            mean_actual = np.mean(q_actual)
            mean_rel_error = (mean_abs_error / mean_actual) * 100 if mean_actual > 0 else 0
            
            # Enhanced fit quality assessment using multiple criteria
            rmse_percent = (rmse / mean_actual) * 100 if mean_actual > 0 else 100
            
            # Multi-criteria fit quality assessment
            if rmse_percent <= 5.0 and mean_rel_error <= 3.0:
                fit_quality = "excellent"
            elif rmse_percent <= 10.0 and mean_rel_error <= 7.0:
                fit_quality = "good"
            elif rmse_percent <= 20.0 and mean_rel_error <= 15.0:
                fit_quality = "fair"
            elif rmse_percent <= 40.0 or r_squared >= 0.3:
                fit_quality = "poor"
            else:
                fit_quality = "very_poor"
            
            # Override for special cases
            if ss_tot == 0:
                fit_quality = "identical_data"
            elif r_squared >= 0.9 and rmse_percent <= 15.0:
                fit_quality = "excellent"
            elif r_squared <= -2.0 and rmse_percent > 30.0:
                fit_quality = "very_poor"
            
            return {
                'sbhp': float(sbhp_opt), 'pi': float(pi_opt), 'r_squared': float(r_squared), 
                'rmse': float(rmse), 'mean_abs_error': float(mean_abs_error), 
                'mean_rel_error': float(mean_rel_error), 'fit_quality': fit_quality, 
                'sse': float(result.fun), 'success': True,
                'q_actual': q_actual.tolist(), 'q_calculated': q_calculated.tolist(), 
                'fbhp': fbhp_data.tolist(), 'dates': group_data['Test Date'].values,
            }
        else:
            return {'success': False, 'message': 'Optimization failed to converge'}

    def process_data(self, data, pbub, sbhp_guess, pi_guess, group_size=5):
        data_sorted = data.sort_values('Test Date').reset_index(drop=True)
        self.data = data_sorted
        self.results = []
        num_groups = len(data_sorted) // group_size
        for i in range(num_groups):
            start_idx = i * group_size
            end_idx = start_idx + group_size
            group_data = data_sorted.iloc[start_idx:end_idx]
            res = self.optimize_group(group_data, pbub, sbhp_guess, pi_guess)
            if res.get('success'):
                res['group_id'] = i+1
                res['start_date'] = group_data['Test Date'].min()
                res['end_date'] = group_data['Test Date'].max()
                self.results.append(res)
        return self.results

    def generate_ipr_curve(self, sbhp, pbub, pi):
        fbhp_range = np.linspace(0.0, float(sbhp), 100)
        q_range = np.array([self.calculate_ipr(f, sbhp, pbub, pi) for f in fbhp_range])
        return fbhp_range, q_range

# End of enhanced class

In [ ]:
# Enhanced Exercise: Compare fit quality assessment methods
if 'df' in globals():
    print("Enhanced Fit Quality Assessment Comparison")
    print("=" * 50)
    
    # Run optimization with different group sizes
    for group_size_test in [3, 5, 10]:
        optimizer = WellTestOptimizer()
        res = optimizer.process_data(df, pbub=4000.0, sbhp_guess=3500.0, pi_guess=1.0, group_size=group_size_test)
        
        if res:
            print(f"\nGroup Size {group_size_test}: {len(res)} results")
            for r in res[:3]:  # Show first 3 results
                print(f"  Group {r['group_id']}: R²={r['r_squared']:.3f}, "
                      f"RMSE={r['rmse']:.1f}, Error%={r['mean_rel_error']:.1f}%, "
                      f"Quality={r['fit_quality']}")
            
            # Count quality categories
            quality_counts = {}
            for r in res:
                quality = r['fit_quality']
                quality_counts[quality] = quality_counts.get(quality, 0) + 1
            print(f"  Quality distribution: {quality_counts}")
        else:
            print(f"\nGroup Size {group_size_test}: No successful results")
    
    print("\nKey Insights:")
    print("• R² can be misleading for engineering applications")
    print("• Error% and RMSE provide practical accuracy measures")
    print("• Multi-criteria assessment gives better engineering decisions")
else:
    print('Upload the CSV (Test-1 welltestdata.csv) and run the data load cell first.')

## Understanding R² vs RMSE for Engineering Applications

**Critical Engineering Insight**: R² alone can be highly misleading for well test analysis!

### Why R² Can Mislead Engineers

The coefficient of determination (R²) measures how well a model explains data variance:
- **R² = 1 - (Model Error / Data Variance)**
- For **low-variance datasets** (stable flow rates), even accurate models get poor R²
- **Engineering reality**: A model predicting 1000±10 bbl/day with 5 bbl/day accuracy is excellent!
- **Statistical reality**: R² could be negative if the model doesn't follow minor fluctuations

### Example Scenario
```
Actual rates: [995, 1000, 1005, 998, 1002] bbl/day (very stable production)
Model predicts: [1000, 1000, 1000, 1000, 1000] bbl/day (constant rate)

RMSE: 3.5 bbl/day (excellent engineering accuracy!)
R²: Could be negative (poor statistical fit)
Engineering Decision: Model is field-ready and highly accurate
```

### Multi-Criteria Assessment Used in This Tool

**Primary Metrics for Engineering Decisions:**
1. **RMSE** (Root Mean Square Error): Absolute accuracy in bbl/day units
2. **Relative Error %**: Practical accuracy as percentage of mean flow rate
3. **R²**: Supplementary statistical correlation measure

**Practical Fit Quality Criteria:**
- **Excellent**: ≤5% relative error (field-ready accuracy)
- **Good**: ≤10% relative error (suitable for most engineering applications)  
- **Fair**: ≤20% relative error (acceptable for screening studies)
- **Poor**: >20% relative error but some correlation exists
- **Very Poor**: >40% relative error with poor correlation

**Recommendation**: Prioritize RMSE and relative error % for engineering decisions. Use R² as supplementary information for statistical model assessment.

In [ ]:
# Practical Demonstration: R² vs RMSE for Engineering Decisions
if 'results' in globals() and len(results) > 0:
    print("Demonstration: Why R² Can Mislead Engineers")
    print("=" * 50)
    
    # Find examples of different R² vs practical accuracy patterns
    for i, res in enumerate(results):
        r2 = res['r_squared']
        rmse = res['rmse']
        error_pct = res['mean_rel_error']
        quality = res['fit_quality']
        
        # Look for interesting cases
        if r2 < 0.5 and error_pct < 10:  # Poor R² but good practical accuracy
            print(f"\n📊 Group {res['group_id']} - R² MISLEADING CASE:")
            print(f"   R² = {r2:.3f} (poor statistical fit)")
            print(f"   RMSE = {rmse:.1f} bbl/day (good engineering accuracy)")
            print(f"   Error% = {error_pct:.1f}% (excellent field accuracy)")
            print(f"   Quality = {quality} (multi-criteria assessment)")
            print(f"   👍 Engineering Decision: This model is field-ready!")
            print(f"   ❌ R²-only Decision: Would reject this excellent model")
            break
    else:
        print("No clear R² vs practical accuracy disconnect found in current results.")
        print("Try different group sizes or datasets to see this phenomenon.")
    
    # Show the old vs new assessment comparison
    print(f"\nComparison of Assessment Methods:")
    print(f"{'Group':<6}{'R²':<8}{'RMSE':<8}{'Error%':<8}{'Old(R²)':<12}{'New(Multi)':<12}")
    print("-" * 60)
    
    for res in results[:5]:  # Show first 5 results
        r2 = res['r_squared']
        rmse = res['rmse']
        error_pct = res['mean_rel_error']
        new_quality = res['fit_quality']
        
        # Simulate old R²-only assessment
        if r2 >= 0.8:
            old_quality = "excellent"
        elif r2 >= 0.5:
            old_quality = "good" 
        elif r2 >= 0.0:
            old_quality = "poor"
        else:
            old_quality = "very_poor"
            
        print(f"{res['group_id']:<6}{r2:<8.3f}{rmse:<8.1f}{error_pct:<8.1f}{old_quality:<12}{new_quality:<12}")
    
    print(f"\nKey Insight: Multi-criteria assessment better identifies field-ready models!")
else:
    print("Run the optimization first to see the R² vs RMSE demonstration.")

## Learning Exercises

Try these exercises to understand how well test analysis works in practice. Work through them at your own pace.

### Basic Analysis Exercises

**Exercise 1 - Group Size Effects**: 
Change the `group_size` parameter (try 1, 3, 5, 10) and observe how the productivity index (PI) and reservoir pressure (SBHP) results change. 
- *Question*: Why do smaller groups give more variable results?
- *Question*: Why do larger groups give more stable but possibly less accurate results?

**Exercise 2 - Bubble Point Impact**: 
Change the `pbub` (bubble point pressure) value (try 3000, 4000, 5000 psi) and see how it affects the flow rate calculations and curve shapes.
- *Question*: How does bubble point pressure affect the IPR curve shape?
- *Question*: Which pressure ranges use which flow models?

**Exercise 3 - Data Quality**: 
Try modifying one data point (make one pressure value unrealistic) and run the analysis again.
- *Question*: How does the system detect bad data?
- *Question*: What warnings do you get, and why are they important?

### Advanced Understanding Exercises

**Exercise 4 - Model Evaluation**: 
Look for wells where R² (statistical fit) is poor but RMSE (engineering accuracy) is good.
- *Question*: When would you trust a model with poor R² but good RMSE?
- *Question*: Which metric matters more for field operations?

**Exercise 5 - Engineering Decisions**: 
Compare the "old" R²-only method with the new multi-criteria assessment.
- *Question*: Find cases where the two methods disagree - which would you trust?
- *Question*: What does "field-ready" accuracy mean for a production engineer?

### What to Include in Your Analysis

For each exercise, document:
- **Your hypothesis**: What do you expect to see?
- **Your procedure**: What you changed and why
- **Your observations**: Numbers, plots, and patterns you notice
- **Engineering interpretation**: What this means for real well operations

## What Data You Need

Your well test data should be in a CSV file with these columns:

- **Test Date**: When each test was conducted (any standard date format)
- **Pwf**: Flowing Bottom Hole Pressure (psi) - the pressure in the wellbore while producing
- **Total Rate**: Liquid production rate (barrels per day) - how much oil/liquid the well produces

**Important**: All pressure and rate values must be positive numbers. The analysis works best when data is arranged chronologically (oldest to newest).

## Quick Start Guide

**How to use this notebook:**

1. **Upload your data**: Use the Files panel on the left to upload your `Test-1 welltestdata.csv` file
2. **Run the setup**: Execute the Import Tools section above (click the play button on each cell)
3. **Load your data**: Run the data loading section below to read your CSV file
4. **Run the analysis**: Execute the optimization section to analyze your well data
5. **View results**: Look at the plots and tables to understand your well's performance

**Tips for getting good results:**
- Try different `group_size` values (3, 5, or 10) to see which gives the most stable results
- Adjust `pbub` (bubble point pressure) based on your reservoir fluid properties
- Check the fit quality metrics to see how reliable your results are

## System Requirements

**What you need:**
- Google Colab (free) - works in any web browser
- Your well test data in CSV format
- About 30-60 minutes for the full analysis

**Note**: Google Colab already has all the mathematical tools we need installed. If you're running this on your own computer, you might need to install: pandas, numpy, matplotlib, scipy, and openpyxl.

## Step 4: Load Your Well Test Data

Upload your well test data file (`Test-1 welltestdata.csv`) using the Files panel on the left side of Colab. 

**To upload a file:**
1. Click the folder icon in the left panel
2. Click "Upload" and select your CSV file
3. Wait for the upload to complete
4. Run the data loading cell below

In [ ]:
# Colab helper: mount Google Drive and copy the example CSV into the session
try:
    from google.colab import drive
    import shutil
    drive.mount('/content/drive')
    drive_src = '/content/drive/My Drive/ColabNotebooks/Test-1 welltestdata.csv'
    if os.path.exists(drive_src):
        shutil.copy(drive_src, './')
        print('Copied Test-1 welltestdata.csv from Drive to session directory.')
    else:
        print('File not found at', drive_src, '\nPlace the CSV in Drive under "My Drive/ColabNotebooks/" or upload via Files pane.')
except Exception as e:
    print('Drive mount not available in this environment or error occurred:', e)

In [ ]:
# Attempt to load the provided example file
csv_path = 'Test-1 welltestdata.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path, parse_dates=['Test Date'])
    df = df.sort_values('Test Date').reset_index(drop=True)
    print(f'Loaded {len(df)} records from {csv_path}')
    display(df.head())
else:
    print(f'File not found: {csv_path} -- upload it to the Colab session or drive and update the path.')

## Step 5: Analyze Your Well Data

Now we'll analyze your well test data to find the best productivity index (PI) and reservoir pressure (SBHP) values. 

**Parameters you can adjust:**
- `pbub`: Bubble point pressure of your reservoir fluid (psi)
- `sbhp_guess`: Your initial estimate of reservoir pressure (psi)  
- `pi_guess`: Your initial estimate of productivity index (bbl/day/psi)
- `group_size`: How many data points to analyze together

The analysis will automatically find the best parameters and tell you how reliable the results are.

In [ ]:
# Parameters (tweak as needed)
pbub = 4000.0
sbhp_guess = 3500.0
pi_guess = 1.0
group_size = 5

# Run optimizer if df exists
if 'df' in globals():
    optimizer = WellTestOptimizer()
    results = optimizer.process_data(df, pbub, sbhp_guess, pi_guess, group_size)
    print(f'Optimization complete. Found results for {len(results)} groups.')
    
    # Convert results to DataFrame for display
    if len(results) > 0:
        results_df = pd.DataFrame(results)
        results_df['start_date'] = pd.to_datetime(results_df['start_date']).dt.strftime('%Y-%m-%d')
        results_df['end_date'] = pd.to_datetime(results_df['end_date']).dt.strftime('%Y-%m-%d')
        
        # Display enhanced results with new metrics
        display_cols = ['group_id', 'start_date', 'end_date', 'sbhp', 'pi', 
                       'r_squared', 'rmse', 'mean_rel_error', 'fit_quality']
        display_df = results_df[display_cols].copy()
        display_df.columns = ['Group', 'Start Date', 'End Date', 'SBHP (psi)', 
                             'PI (bbl/day/psi)', 'R²', 'RMSE', 'Error %', 'Fit Quality']
        
        # Round numerical columns for better display
        display_df['SBHP (psi)'] = display_df['SBHP (psi)'].round(0)
        display_df['PI (bbl/day/psi)'] = display_df['PI (bbl/day/psi)'].round(3)
        display_df['R²'] = display_df['R²'].round(3)
        display_df['RMSE'] = display_df['RMSE'].round(1)
        display_df['Error %'] = display_df['Error %'].round(1)
        
        print("\nOptimization Results with Enhanced Fit Quality Assessment:")
        print("=" * 80)
        display(display_df)
        
        print("\nFit Quality Assessment Notes:")
        print("• RMSE: Root Mean Square Error in flow rate units (bbl/day)")
        print("• Error %: Mean relative error as percentage of actual flow rates")
        print("• R²: Coefficient of determination (can be negative for poor fits)")
        print("• Fit Quality: Multi-criteria assessment prioritizing practical accuracy")
        
        # Summary statistics
        excellent_count = sum(1 for r in results if r['fit_quality'] == 'excellent')
        good_count = sum(1 for r in results if r['fit_quality'] == 'good')
        total_count = len(results)
        success_rate = ((excellent_count + good_count) / total_count) * 100 if total_count > 0 else 0
        
        print(f"\nSummary: {excellent_count} excellent + {good_count} good = {success_rate:.1f}% success rate")
        
    else:
        print("No successful optimizations found. Check data quality and parameters.")
else:
    print('No data loaded; please upload the CSV and run this cell again.')

## Step 6: View Your Results

These plots help you understand your well's performance and how reliable the analysis is.

**Plot 1 - Parameter Trends**: Shows how productivity index and reservoir pressure change over time, with colors indicating result quality.

**Plot 2 - Model Accuracy**: Compares actual flow rates to calculated values. Points close to the red line mean the model is accurate.

**Plot 3 - IPR Curves**: Shows the theoretical relationship between pressure and flow rate for your well, with your actual test data overlaid.

In [ ]:
if 'results' in globals() and len(results) > 0:
    results_df = pd.DataFrame(results)
    
    # Enhanced Plot 1: Parameter trends with fit quality
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
    
    # SBHP trend
    colors = ['green' if fq in ['excellent', 'good'] else 'orange' if fq == 'fair' else 'red' 
             for fq in results_df['fit_quality']]
    ax1.scatter(results_df['group_id'], results_df['sbhp'], c=colors, s=80, alpha=0.7, edgecolor='black')
    ax1.plot(results_df['group_id'], results_df['sbhp'], 'b-', alpha=0.5, linewidth=1)
    ax1.set_ylabel('SBHP (psi)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Optimized Parameters by Group (Color = Fit Quality)', fontsize=14, fontweight='bold')
    
    # PI trend
    ax2.scatter(results_df['group_id'], results_df['pi'], c=colors, s=80, alpha=0.7, edgecolor='black')
    ax2.plot(results_df['group_id'], results_df['pi'], 'r-', alpha=0.5, linewidth=1)
    ax2.set_ylabel('PI (bbl/day/psi)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Error percentage trend
    ax3.scatter(results_df['group_id'], results_df['mean_rel_error'], c=colors, s=80, alpha=0.7, edgecolor='black')
    ax3.plot(results_df['group_id'], results_df['mean_rel_error'], 'purple', alpha=0.5, linewidth=1)
    ax3.set_ylabel('Error %', fontsize=12)
    ax3.set_xlabel('Group ID', fontsize=12)
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=10, color='orange', linestyle='--', alpha=0.7, label='10% threshold')
    ax3.axhline(y=5, color='green', linestyle='--', alpha=0.7, label='5% threshold')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

    # Enhanced Plot 2: Actual vs Calculated with metrics for best group
    best_group_idx = np.argmin([r['mean_rel_error'] for r in results])
    selected = results[best_group_idx]
    
    q_act = np.array(selected['q_actual'])
    q_calc = np.array(selected['q_calculated'])
    
    plt.figure(figsize=(8, 6))
    plt.scatter(q_act, q_calc, alpha=0.8, s=80, edgecolor='black', linewidth=1.5)
    
    # Perfect fit line
    mn = min(q_act.min(), q_calc.min()) * 0.95
    mx = max(q_act.max(), q_calc.max()) * 1.05
    plt.plot([mn,mx], [mn,mx], 'r--', linewidth=2, label='Perfect Fit')
    
    # ±10% error bands
    plt.fill_between([mn,mx], [mn*0.9, mx*0.9], [mn*1.1, mx*1.1], 
                    alpha=0.1, color='green', label='±10% Error Band')
    
    plt.xlabel('Actual Rate (bbl/day)', fontsize=12)
    plt.ylabel('Calculated Rate (bbl/day)', fontsize=12)
    plt.title(f'Actual vs Calculated (Group {selected["group_id"]} - Best Fit)\n'
             f'R² = {selected["r_squared"]:.3f}, RMSE = {selected["rmse"]:.1f}, '
             f'Error% = {selected["mean_rel_error"]:.1f}%', fontsize=12, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.axis('equal')
    
    # Add text box with metrics explanation
    textstr = f'Fit Quality: {selected["fit_quality"].title()}\nThis model explains the data with {selected["mean_rel_error"]:.1f}% average error'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    
    plt.tight_layout()
    plt.show()

    # Enhanced Plot 3: IPR curves overlay with improved legend
    plt.figure(figsize=(12, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(results)))
    
    for i, res in enumerate(results):
        fb, ql = optimizer.generate_ipr_curve(res['sbhp'], pbub, res['pi'])
        
        # Line style based on fit quality
        if res['fit_quality'] in ['excellent', 'good']:
            linestyle = '-'
            alpha = 0.8
        else:
            linestyle = '--'
            alpha = 0.5
            
        plt.plot(ql, fb, color=colors[i], linestyle=linestyle, alpha=alpha, linewidth=2,
                label=f"Group {res['group_id']} ({res['fit_quality']}, {res['mean_rel_error']:.1f}% err)")
        
        # Plot measured data points
        plt.scatter(res['q_actual'], res['fbhp'], color=colors[i], 
                   edgecolor='black', alpha=0.8, s=60, linewidth=1)
    
    plt.xlabel('Liquid Flow Rate (bbl/day)', fontsize=12)
    plt.ylabel('FBHP (psi)', fontsize=12)
    plt.title('IPR Curves with Measured Data\n(Solid lines = good fits, Dashed = poor fits)', 
             fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Improved legend positioning
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.tight_layout()
    plt.show()
    
    # Summary metrics table
    summary_data = []
    for res in results:
        summary_data.append({
            'Group': res['group_id'],
            'SBHP': f"{res['sbhp']:.0f}",
            'PI': f"{res['pi']:.3f}",
            'R²': f"{res['r_squared']:.3f}",
            'RMSE': f"{res['rmse']:.1f}",
            'Error%': f"{res['mean_rel_error']:.1f}",
            'Quality': res['fit_quality']
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\nDetailed Results Summary:")
    print("=" * 70)
    display(summary_df)
    
else:
    print('No optimization results to plot. Run the optimization cell first.')

## Step 7: Save Your Results (Optional)

You can save your analysis results to an Excel file to use in reports or share with others.

In [ ]:
# Example: save results to Excel locally (or to Colab drive)
if 'results_df' in globals():
    out_name = 'well_test_results.xlsx'
    results_df.to_excel(out_name, index=False)
    print(f'Saved results to {out_name}')
else:
    print('No results to save.')

---
## Summary: What You've Learned

**Analysis Capabilities:**
- **Multi-criteria evaluation**: The system uses multiple measures (RMSE, relative error %, and R²) to assess how good your results are
- **Automatic quality checking**: Built-in validation helps identify data problems and unreliable results
- **Robust calculations**: The analysis handles challenging data and provides reliable engineering parameters
- **Visual understanding**: Multiple plots help you see patterns and evaluate result quality

**Key Engineering Insights:**
- **R² limitations**: Statistical measures like R² can sometimes mislead engineers, especially with stable production data
- **Practical accuracy**: RMSE and relative error % often give better measures of real-world model usefulness
- **Engineering judgment**: Understanding when to trust models despite imperfect statistical fits is crucial for field applications
- **Quality assessment**: Multiple criteria help identify field-ready models that single-metric approaches might reject

**What This Means for Field Work:**
- **Better decisions**: Multi-criteria assessment helps you choose models suitable for field development
- **Quality understanding**: You can now evaluate when results are reliable enough for engineering decisions
- **Practical focus**: The analysis prioritizes engineering accuracy over statistical perfection
- **Real-world application**: Results include guidance on when models are ready for field implementation

**Further Learning:**
- Try the analysis with different well datasets to see how reservoir characteristics affect results
- Experiment with various bubble point pressures to understand flow regime transitions
- Practice interpreting the quality metrics to develop engineering judgment for model evaluation